In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
from keras import layers
import keras

In [2]:
from keras import layers
import keras

In [34]:
#Inisialisasi ImageDataGenerator
train_eyes_aug = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                                 validation_split=0.2)

train_mouth_aug = tf.keras.preprocessing.image.ImageDataGenerator

#Data Splitting untuk data mata
train_eyes = train_eyes_aug.flow_from_directory('Data/train1/',
                                                target_size=(225, 225),
                                                batch_size=32,
                                                color_mode = 'grayscale',
                                                class_mode='binary',
                                                subset = 'training')

valid_eyes = train_eyes_aug.flow_from_directory('Data/train1/',
                                                target_size=(225, 225),
                                                batch_size=16,
                                                color_mode = 'grayscale',
                                                class_mode='binary',
                                                subset = 'validation')

#Data Splitting untuk data mulut
train_mouth = train_mouth_aug.flow_from_directory('Data/train2/',
                                                target_size=(225, 225),
                                                batch_size=32,
                                                color_mode = 'grayscale',
                                                class_mode='binary',
                                                subset ='training')

valid_mouth = train_mouth_aug.flow_from_directory('Data/train2/',
                                                target_size=(225, 225),
                                                batch_size=16,
                                                color_mode = 'grayscale',
                                                class_mode='binary',
                                                subset ='validation')

Found 1158 images belonging to 2 classes.
Found 288 images belonging to 2 classes.
Found 1158 images belonging to 2 classes.
Found 288 images belonging to 2 classes.


In [31]:
input_eyes = tf.keras.Input(shape = (225,225,1), name='eyes_input')
input_mouth = tf.keras.Input(shape = (225,225,1 ), name = 'mouth_input')

x1 = tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu', padding = 'valid')(input_eyes)
x1 = tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), activation = 'relu', padding = 'valid')(x1)
x1 = tf.keras.layers.MaxPool2D()(x1)
x1 = tf.keras.layers.Flatten()(x1)

x2 = tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu', padding = 'valid')(input_mouth)
x2 = tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), activation = 'relu', padding = 'valid')(x2)
x2 = tf.keras.layers.MaxPool2D()(x2)
x2 = tf.keras.layers.Flatten()(x2)

comb = tf.keras.layers.Concatenate()([x1, x2])

fc = tf.keras.layers.Dense(64)(comb)

output_eye = tf.keras.layers.Dense(1, activation='sigmoid', name = 'out_eyes')(fc)
output_mouth = tf.keras.layers.Dense(1, activation='sigmoid', name = 'out_mouth')(fc)

model = tf.keras.Model(inputs = [input_eyes, input_mouth],
                    outputs = [output_eye, output_mouth])


In [32]:
model.compile(optimizer = keras.optimizers.Adam(),
              loss = {
                  'out_eyes' : keras.losses.BinaryCrossentropy(),
                  'out_mouth' : keras.losses.BinaryCrossentropy(),
                  },
              metrics = {
                  'out_eyes' : keras.metrics.BinaryAccuracy(),
                  'out_mouth' : keras.metrics.BinaryAccuracy(),
                  }
              )

In [37]:
model.fit([train_eyes, train_mouth], 
          validation_data = [valid_eyes , valid_mouth],
          epochs = 10)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'keras.src.preprocessing.image.DirectoryIterator'>"}), <class 'NoneType'>

SyntaxError: incomplete input (1399920271.py, line 1)